<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# change default style figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,sklearn,matplotlib

Ethen 2018-12-05 19:39:27 

CPython 3.6.4
IPython 6.4.0

numpy 1.14.1
pandas 0.23.0
sklearn 0.19.1
matplotlib 2.2.2


In [2]:
import os
import zipfile

def download(url, filename):
    with open(filename, 'wb') as file:
        response = requests.get(url)
        file.write(response.content)

# we'll download the data to DATA_DIR location
DATA_DIR = './datasets/'
URL = 'http://nlp.stanford.edu/data/glove.twitter.27B.zip'
filename = os.path.join(DATA_DIR, 'glove.twitter.27B.zip')
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

if not os.path.exists(filename):
    download(URL, filename)

In [3]:
def get_train_test_data(filename, dimension=25, test_size=0.2, random_state=1234):
    """
    dimension : int, {25, 50, 100, 200}, default 25
        The dataset contains embeddings of different size.
    """
    with zipfile.ZipFile(filename) as f:
        X = []
        zip_filename = 'glove.twitter.27B.{}d.txt'.format(dimension)
        for line in f.open(zip_filename):
            # remove the first index, id field and only get the vectors
            vector = np.array([float(x) for x in line.strip().split()[1:]])
            X.append(vector)

        X_train, X_test = train_test_split(
            np.array(X), test_size=test_size, random_state=random_state)

    # we can downsample for experimentation purpose
    # X_train = X_train[:50000]
    # X_test = X_test[:10000]
    return X_train, X_test


X_train, X_test = get_train_test_data(filename)
print('training data shape: ', X_train.shape)
print('testing data shape: ', X_test.shape)

training data shape:  (954811, 25)
testing data shape:  (238703, 25)


In [9]:
def make_random_model():
    m = LOPQModel(V=5, M=4, subquantizer_clusters=10)
    m.fit(np.random.RandomState(42).rand(200, 8), n_init=1)
    return m

m = make_random_model()
m

In [12]:
m.predict(np.random.RandomState(42).rand(200, 8))

ValueError: operands could not be broadcast together with shapes (100,8) (5,4) 

In [4]:
from lopq import LOPQModel, LOPQSearcher

# Define a model and fit it to data
model = LOPQModel(V=8, M=4)
model.fit(X_train)

# Compute the LOPQ codes for a vector
code = model.predict(X_train)
code

ValueError: array split does not result in an equal division

In [ ]:
# Create a searcher to index data with the model
searcher = LOPQSearcher(model)
searcher.add_data(data)

# Retrieve ranked nearest neighbors
nns = searcher.search(x, quota=100)